## test_predからpseudo-labelを作成するnotebook --

In [75]:
import pandas as pd
import numpy as np

from config import *

In [76]:
run_id_list = [
    "bert_large",
]

In [77]:
pseudo_class1_idlist = pd.DataFrame()
for run_id in run_id_list:
    test_df = pd.read_feather(f"{output_root}{run_id}/test_df.feather")
    pseudo_class1_idlist = pd.concat([pseudo_class1_idlist, test_df[test_df["model_oof_class_1"] > 0.9]["id"]])

In [78]:
pseudo_class1_idlist

,0
372,1ea50e5d9
409,2165f57a5
628,30eb0dcbe
653,329722989
1148,5b2799da6
1153,5b4d1ebb6
1295,662cb3a59
1329,689da439e
1374,6b48e32bc
1424,7027000f1


In [79]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, T5Tokenizer
tokenizer1 = AutoTokenizer.from_pretrained("nlp-waseda/roberta-large-japanese-seq512")
tokenizer2 = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")

In [80]:
from pyknp import Juman, BList, KNP

# avoid juman warnings --
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def juman_parse(text):
    words = ""
    jumanpp = Juman()
    result = jumanpp.analysis(text)
    for mrp in result.mrph_list():
        words += (mrp.midasi + " ")
    return words[:-1]  # last " " omit by [:-1] -- 

In [81]:
t = juman_parse("人望はないが珍棒はある。")

In [82]:
tokenizer1.convert_ids_to_tokens(tokenizer1.encode(t))

['[CLS]', '▁人', '望', '▁は', '▁ない', '▁が', '▁珍', '棒', '▁は', '▁ある', '▁。', '[SEP]']

In [83]:
tokenizer2.convert_ids_to_tokens(tokenizer2.encode(t))

['▁',
 '人',
 '望',
 '▁',
 'は',
 '▁',
 'ない',
 '▁',
 'が',
 '▁',
 '珍',
 '棒',
 '▁',
 'は',
 '▁ある',
 '▁',
 '。',
 '</s>']

In [84]:
train_df = pd.read_feather("./output/bert_baseline/train_df.feather")

In [85]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

train_df["clean_text"] = train_df["clean_text"].parallel_map(lambda x: juman_parse(x))

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [86]:
train_df

,index,id,source,text,label,clean_text,kfold,model_oof_class_0,model_oof_class_1,model_pred
0,0,80074aa43,news4vip,まともに相手されてない人との関係なんて\nそんな大事にするものか？,0.0,まともに 相手 さ れて ない 人 と の 関係 なんて そんな 大事に する もの か ?,0,0.997844,0.002333,0.0
1,1,6378fea6b,livejupiter,最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!,0.0,最近 は アヘアヘ QS マン や ない ? イイ ! ( ・ ∀ ・ ) + 1 - ...,4,0.995219,0.005035,0.0
2,2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ\n甘えるな,1.0,日本 人 と して 生まれて も 無能な 低 学歴 って 分かったら 日本 人 の 権利 剥...,3,0.309533,0.693114,1.0
3,3,e76638295,livejupiter,よくよく思えば川上は配布にしたらとんでもなく有能だよな\nガチャから引いたら圧倒的歓喜レベルやで,0.0,よくよく 思えば 川上 は 配布 に したら とんでもなく 有能だ よ な ガチャ から 引...,1,0.996962,0.003418,0.0
4,4,51e4036bf,newsplus,押井は原作レイプの専門家だから\n原作マンガの真意を誤解させることに関してはプロだが\nそれ...,0.0,押井 は 原作 レイプ の 専門 家 だ から 原作 マンガ の 真意 を 誤解 さ せる ...,3,0.996705,0.003385,0.0
...,...,...,...,...,...,...,...,...,...,...
5251,5251,42b5f86b0,news4vip,車じゃなくてもよくない？\nケーブル網を張り巡らせてリフトみたいなのを付けるとか,0.0,車 じゃ なくて も よく ない ? ケーブル 網 を 張り巡らせて リフト みたいな の ...,4,0.997860,0.002067,0.0
5252,5252,402ce15d9,livejupiter,左やぞ？\n出すなら下水流,0.0,左 や ぞ ? 出す なら 下水流,0,0.997378,0.002209,0.0
5253,5253,0739a9fcb,newsplus,日本によって、けんけんガクガクの議論を持たらされた韓国は被害者\n\n日本人がしっかり考えな...,0.0,日本 に よって けんけん ガクガク の 議論 を 持た ら さ れた 韓国 は 被害 者 ...,3,0.942740,0.052718,0.0
5254,5254,d496c7dc8,news4vip,ゴムボート買って、沖まで漕いで行ったら？\n魚の血を塗っておけばサメが食べてくれるよ,0.0,ゴム ボート 買って 沖 まで 漕いで 行ったら ? 魚 の 血 を 塗って おけば サメ ...,3,0.991694,0.009647,0.0
